In [1]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [2]:
# get data
data = pd.read_csv('Level_3a_NonPath_SP1_ZyMo_PSL_1_2_179.csv') 

# split data
X=data.iloc[:,1:51]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

         Xc1.A      Xc1.R     Xc1.N  ...  Xc2.lambda28  Xc2.lambda29  Xc2.lambda30
0    14.767060  10.829177  7.383530  ...      0.025877      0.025830      0.026292
1    11.728905   8.138424  3.351116  ...      0.025056      0.025025      0.026281
2     8.319776   2.773259  1.663955  ...      0.022828      0.023152      0.024468
3    10.255686   3.778411  2.428978  ...      0.023815      0.022707      0.024686
4     6.745884   2.698354  0.809506  ...      0.024331      0.024572      0.023173
..         ...        ...       ...  ...           ...           ...           ...
174   1.942887   3.157192  0.971444  ...      0.025157      0.024279      0.024645
175   3.204075   3.450542  1.478804  ...      0.022274      0.026769      0.022444
176  13.834749   3.132396  2.610330  ...      0.024473      0.023891      0.023542
177   4.081263   3.361040  0.240074  ...      0.026715      0.026849      0.025076
178   6.898783   3.695777  2.956621  ...      0.024746      0.026312      0.023982

[17

In [3]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [4]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

#sm = SMOTE(random_state=45)
#x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(X_ld, yy, test_size=0.2,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(50, 2 - 1) = 1 components.
  ChangedBehaviorWarni

In [5]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(2 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 50)             10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 128)            91648     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
Total params: 151,586
Trainable params: 151,586
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
5/5 [==============================] - 35s 744ms/step - loss: 0.6913 - accuracy: 0.7663 - val_loss: 0.6860 - val_accuracy: 0.8333
Epoch 2/1000
5/5 [==============================] - 0s 16ms/step - loss: 0.6839 - accuracy: 0.9061 - val_loss: 0.6777 - val_accuracy: 0.8611
Epoch 3/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.6737 - accuracy: 0.8995 - val_loss: 0.6663 - val_accuracy: 0.8611
Epoch 4/1000
5/5 [==============================] - 0s 15ms/step - loss: 0.6636 - accuracy: 0.9095 - val_loss: 0.6507 - val_accuracy: 0.8889
Epoch 5/1000
5/5 [==============================] - 0s 15ms/step - loss: 0.6418 - accuracy: 0.9475 - val_loss: 0.6286 - val_accuracy: 0.9167
Epoch 6/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.6222 - accuracy: 0.9337 - val_loss: 0.5986 - val_accuracy: 0.9167
Epoch 7/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.5902 - accuracy: 0.9375 - val_loss: 0.5583 - val_accuracy: 0.9167
Epoch 8/100

In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.9440559440559441
test-acc = 0.9166666666666666
train-CFR =               precision    recall  f1-score   support

           0       1.00      0.64      0.78        22
           1       0.94      1.00      0.97       121

    accuracy                           0.94       143
   macro avg       0.97      0.82      0.87       143
weighted avg       0.95      0.94      0.94       143

test-CFR =               precision    recall  f1-score   support

           0       1.00      0.50      0.67         6
           1       0.91      1.00      0.95        30

    accuracy                           0.92        36
   macro avg       0.95      0.75      0.81        36
weighted avg       0.92      0.92      0.90        36

